# Value at Risk (VaR)

The _value at risk_ of a portfolio, or _VaR_ is a measure that allows us to quantify potential losses in an investment using basic statistical tools.

In simple terms, _VaR_ provides an estimate of the maximum loss an investment or portfolio might experience over a defined period, given a certain confidence level. For example, a 95% _VaR_ with a one-day time horizon estimates the potential losses that will not be exceeded with 95% confidence in a single day.
is typically expressed as a monetary value or a percentage of the investment or portfolio value. For example, a VaR of €100,000 means there is a 95% probability that the maximum loss over the defined time horizon will not exceed €100,000.

$\displaystyle \text{VaR} = P \cdot (-z) \cdot \sigma$

Where $P$ is the total value of the portfolio on a given date, $\sigma$ is the standard deviation of returns, and $-z$ represents the number of standard deviations corresponding to the desired confidence level, obtained using the inverse cumulative distribution function (CDF) of the chosen distribution.

### Data Preparation

In [ ]:
from sample_data import stock_prices
aapl = stock_prices("aapl")
msft = stock_prices("msft")
prices = aapl.Close
prices[:2]

## Basic calculations step by step

### 1. Daily returns

In [ ]:
import numpy as np

def daily_returns(closes):
    return np.log(closes) - np.log(closes.shift(1))

returns = daily_returns(prices)
returns[:3]

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
returns.plot.hist(bins=100, color='salmon')

You could also obtain these results by using panda's pct_change method:

In [ ]:
prices.pct_change()[:3]

### 2. Z-Score

$\displaystyle z = \frac{x - \mu}{\sigma}$

Where $x$ is the individual data point, $\mu$ corresponds to the distribution mean and $\sigma$ to the standard deviation

`ppf` function returns a z-score that has 95% of the data above.

In [ ]:
from scipy.stats import norm
z_score = norm.ppf(0.95) # loc:0 (µ), scale: 1 (σ)
z_score

### 3. Portfolio value on 02/01/2015

In [ ]:
shares = 1000
portfolio_value = shares * prices['2015-1-02']
portfolio_value

### 4. Value at Risk

In [ ]:
value_at_risk = portfolio_value * z_score * returns.std()
value_at_risk

Let’s recall the interpretation of what that value means: with 95% confidence, losses would not exceed 805.65 USD the next day. In other words, there is a 5% probability of losing 805.65 USD.

We can repeat calculations to have every day results

In [ ]:
multiplier = (z_score * returns.std())
many_vars = (prices * shares) * multiplier
many_vars['2014-12-30': '2015-01-05']

## Method 2: Historical Data

In [ ]:
def estimate_historical(returns, probability, portfolio_size):
    if (probability > 0.5):
        probability = 1 - probability
    return -(returns.quantile(probability) * portfolio_size)
    
estimate_historical(returns.dropna(), 0.05, portfolio_value)

Given a portfolio size of \\$27,332.50, there is a 5% probability that losses will exceed \\$742.76 over the specified time horizon.

In other words, based on historical data and assuming that returns follow a normal distribution, there is a 5% chance that the portfolio could lose more than $742.76 in that period.

The function that calculates the VaR first checks if the provided probability is greater than 0.5. If it is, it adjusts the probability by subtracting it from 1. This is done because VaR focuses on losses, which are in the lower tail of the return distribution, if someone enters 0.95 (which would refer to gains), the function adjusts it to 0.05 to correctly focus on the worst 5% of cases.

Another way of doing it would be:

In [ ]:
def estimate_historical_normal_distribution(returns, probability, portfolio_size):
    if (probability > 0.5):
        probability = 1 - probability
    expected_return = returns.mean()
    std_dev = returns.std()
    return -(norm.ppf(probability, expected_return, std_dev) * portfolio_size)

estimate_historical_normal_distribution(returns.dropna(), 0.05, portfolio_value)

## Method 3: Variance-Covariance Method
 
Given a portfolio of $P$ size with a confidence level of $c$, and considering daily returns. This method does not use actual returns, it assumes our returns data follows a normal distribution.
 
Daily VaR for a single asset is $P - (P \cdot (\alpha(1-c) + 1))$
 
Where α is the inverse of the cumulative distribution function of a normal distribution with mean μ and standard deviation σ.

In [ ]:
def var_covar(P, c, mu, sigma):
    alpha = norm.ppf(1-c, mu, sigma)
    return P - P*(alpha + 1)

var_covar(portfolio_value, 0.95, returns.mean(), returns.std())